# TensorFlow: DNN using MNIST dataset

Demonstrates creating DNN to accomplish image classification with Conv2D and MaxPool2D layers.
Model is trained on fashion MNIST dataset.

In [ ]:
import os
import matplotlib.pyplot as plt

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

print(f"TF Version: {tf.__version__}")
print(f"TF Devices: {[d.device_type for d in tf.config.list_physical_devices()]}")

In [ ]:
(train_ds, val_ds, test_ds), ds_info = tfds.load("fashion_mnist",
    split=["train[:80%]", "train[80%:]", "test"],
    with_info=True,
    as_supervised=True)

In [ ]:
_ = tfds.show_examples(train_ds.take(3), ds_info)

In [ ]:
train_ds = (train_ds
            .shuffle(buffer_size=1000)
            .batch(32)
            .prefetch(tf.data.experimental.AUTOTUNE))

test_ds = (test_ds
           .batch(32)
           .prefetch(tf.data.experimental.AUTOTUNE))

val_ds = (val_ds
          .batch(32)
          .prefetch(tf.data.experimental.AUTOTUNE))

In [ ]:
# Define model
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(28, 28, 1)),
    tf.keras.layers.Rescaling(1.0 / 255.0),
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(16, kernel_size=(3, 3), activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [ ]:
# Plot model architecture
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# Compile model
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        patience=5,
        min_delta=1e-2,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.TensorBoard(
        log_dir="logs",
        histogram_freq=0,
        embeddings_freq=0,
        update_freq="epoch"
    )
]

history = model.fit(
    train_ds,
    epochs=20,
    callbacks=callbacks,
    validation_data=val_ds,
    verbose=2)

In [ ]:
model.evaluate(test_ds)

In [ ]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]

In [ ]:
xs = range(1, len(loss) + 1)
plt.figure(figsize=(9,3))
plt.subplot(121)
plt.plot(xs, loss, "b", label="Training Loss")
plt.plot(xs, val_loss, "r", label="Validation Loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.subplot(122)
plt.plot(xs, accuracy, "b", label="Training Accuracy")
plt.plot(xs, val_accuracy, "r", label="Validation Accuracy")
plt.title("Training and Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()